In [3]:
from post_parser_record import PostParserRecord
post_reader = PostParserRecord("Posts_Coffee.xml")
!pip3 install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nicdr_nactbrj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicdr_nactbrj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#creating index
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
stop_words.append('p')

inverted_index = {}
word_count_index = {} #{document_id,word_count}
average_doc_word_count = 0
total_document_count = 0

#updates index for each document
def update_index(tokens, document_id):
  global average_doc_word_count
  global total_document_count
  #counts total document count
  total_document_count += 1
  
  average_doc_word_count += len(tokens)
  word_count_index[document_id] = len(tokens)
  #runs through all tokens and indexed accordingly
  for token in tokens:
    #if token not in stopwords.words('english'):
      if token in inverted_index:
        if document_id in inverted_index[token]:
          inverted_index[token][document_id] += 1
        else:
          inverted_index[token][document_id] = 1
      else:
        inverted_index[token] = {document_id: 1}

#creates tokens for given text
def create_tokens_for_text(text):
  temp_tokens = word_tokenize(re.sub(r'[^\w\s]', ' ', text))
  tokens = [i for i in temp_tokens if i not in stop_words]
  return tokens

#indexes questions
print("indexing questions...")
for question_id in post_reader.map_questions:
  question = post_reader.map_questions[question_id]
  title = question.title.lower().strip()
  body = question.body.lower().strip()
  text = title + " " + body
  tokens = create_tokens_for_text(text)
  update_index(tokens, question_id)
  
print("complete")

#indexes answers
print("indexing answers...")
for answer_id in post_reader.map_just_answers:
  answer = post_reader.map_just_answers[answer_id]
  text = answer.body.lower().strip()
  tokens = create_tokens_for_text(text)
  update_index(tokens, answer_id)
average_doc_word_count = average_doc_word_count / total_document_count
print("Complete")
#print(inverted_index)


indexing questions...
complete
indexing answers...
Complete


In [12]:
  
#run to simply initalize this function for use 
from re import A
import math
#searches index for document matches
def BM25(query,query_num):
  bm25 = {}
  k = 1.2
  b = .75
  tf = 0
  idf = 0
  for word in query:
    total_word_count = 0
    #counts total words
    for document in inverted_index[word]:
      total_word_count += inverted_index[word][document]
    #continues
    for document in inverted_index[word]:
      #saves parts for BM25 calculation
      tf = (inverted_index[word][document]/word_count_index[document])
      idf = (math.log(((total_document_count)/(1+total_word_count))))
      dl_adl = word_count_index[document]/average_doc_word_count

      #uses saved values to calculate BM25
      if document not in bm25:
        bm25[document] = tf * idf
        bm25[document] = idf * ((k+1*tf)/((tf+k)*(1.0 - b + (b * dl_adl))))
      else:
        bm25[document] += idf * ((k+1*tf)/((tf+k)*(1.0 - b + (b * dl_adl))))

  #sorts results largest first
  result = sorted(bm25.items(), key=lambda x: x[1], reverse=True)
  i = 0
  for items in result:
    #makes sure only 5 of each pass are printed
    if i <= 4:
      print("Document: "+str(items[0])+"\tScore: "+str(items[1]))
      #to print qrel
      #print("A"+str(query_num)+"\t0\t"+str(items[0])+"\t")
      #to print run_file
      #print("A"+str(query_num)+"\tQ0\t"+str(items[0])+"\t"+str(i+1)+"\t"+str("%.02f"%items[1])+"\tSTANDARD")
      i += 1
    else:
      break
################################################################################
#Querys
#simply add Querys to list 
querys = ["espresso","turkish coffee","making a decaffeinated coffee","can I use the same coffee grounds twice?"]

query_num = 0
#scans through each query
for temp_query in querys:
  temp_query = temp_query.lower().strip()
  temp_tokens = word_tokenize(re.sub(r'[^\w\s]', '', temp_query))
  query_tokens = [i for i in temp_tokens if i not in stop_words]
  query_num += 1
  print("Searching: "+temp_query)
  #uses function to search index and print final result
  BM25(query_tokens,query_num)
  

Searching: espresso
Document: 3904	Score: 2.0784213678473935
Document: 4404	Score: 1.8639027427470725
Document: 95	Score: 1.7724342457061208
Document: 4486	Score: 1.7724342457061208
Document: 2950	Score: 1.7724342457061208
Searching: turkish coffee
Document: 4486	Score: 7.753714720306248
Document: 3369	Score: 7.39101089326847
Document: 5182	Score: 5.327411870482642
Document: 2879	Score: 4.805761773850875
Document: 5094	Score: 4.762993780825598
Searching: making a decaffeinated coffee
Document: 3225	Score: 10.594121159754671
Document: 3293	Score: 8.720526258337124
Document: 204	Score: 8.292578269441174
Document: 2897	Score: 6.658736093340034
Document: 97	Score: 6.159593998918753
Searching: can i use the same coffee grounds twice?
Document: 1749	Score: 11.011258712345983
Document: 2683	Score: 9.607546434699618
Document: 3966	Score: 9.43542627425028
Document: 5582	Score: 9.41040956537419
Document: 3266	Score: 7.801729802826547
